In [ ]:
%matplotlib inline
from matplotlib import pyplot
import geopandas
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.affinity import scale
from shapely.affinity import translate
from shapely.affinity import rotate
import numpy as np

from tile_units import TileUnit
from tile_map import Tiling

# Exploring possibilities in tiling

In [ ]:
ak = geopandas.read_file("../data/imd-auckland-2018.gpkg")

In [ ]:
import copy

def gridify(gs, d = 3):
    return geopandas.GeoSeries.from_wkt(gs.to_wkt(rounding_precision = d))

## Cairo
The one true tiling.

In [ ]:
d = 400
tile_unit = TileUnit(spacing = d, crs = 2193)
tile = tile_unit.tile.geometry[0]

In [ ]:
subtile = rotate(translate(tile, d, d), 30, origin = (0, 0))
subtiles = [rotate(subtile, a, origin = (0, 0)) 
            for a in range(0, 360, 90)]
subtiles = geopandas.GeoDataFrame(
    data = {"element_id": list("abcd")}, crs = 2193,
    geometry = geopandas.GeoSeries(subtiles))
subtiles = tile_unit.tile.overlay(subtiles)

subtiles1 = subtiles.geometry.translate(-d*0.999, -d+0.999)
subtiles2 = subtiles1.scale(-1, 1, origin = (0, 0))
subtiles3 = subtiles2.scale(1, -1, origin = (0, 0))
subtiles4 = subtiles1.scale(1, -1, origin = (0, 0))

cairo_tile = geopandas.GeoSeries(
    list(subtiles1) + list(subtiles2) + list(subtiles3) + list(subtiles4))
cairo_unit = geopandas.GeoDataFrame(
    data = {"element_id": list("abcdcbadcdabadcb")}, crs = 2193,
    geometry = cairo_tile
)
cairo_unit.geometry = cairo_unit.geometry.scale(
    0.5/0.999, 0.5/0.999, origin = (0, 0))
cairo_unit = cairo_unit.dissolve(by = "element_id", as_index = False)
cairo_unit.plot(column = "element_id", edgecolor = "w", cmap = "viridis")


In [ ]:
tile_unit.elements = cairo_unit
tiling = Tiling(tile_unit, ak, id_var = "DZ2018")
t30 = tiling.get_tiled_map(rotation = 0)

In [ ]:
fig = pyplot.figure(figsize = (24, 15))
ax = fig.add_subplot(111)

strands = set(t30.element_id)
cols = [x for x in t30.columns if "Rank" in x]
cmaps = ("Reds", "Greys", "Blues", "Greens")

for strand, col, cmap in zip(strands, cols, cmaps):
    t30[t30.element_id == strand].plot(ax = ax, column = col, cmap = cmap)

## Wobbly-sided 'Escherian' squares

In [ ]:

d = 350

tile_base = Polygon([(-d, -d), (d, -d), (d, d), (-d, d)])
tile_base = geopandas.GeoDataFrame(
    geometry = geopandas.GeoSeries([tile_base]), crs = ak.crs)

In [ ]:
x1 = [x for x in range(-d, d, 3)]
y1 = [-d - d/2 * np.sin(x/d * np.pi) for x in x1]
edge1 = [Point(x, y) for x, y in zip(x1, y1)]
edge3 = [translate(p, 0, 2*d) for p in edge1]
edge3.reverse()

y2 = x1.copy()
x2 = [ d + d/2 * np.sin(y/d * np.pi) for y in y2]
edge2 = [Point(x, y) for x, y in zip(x2, y2)]
edge4 = [translate(p, -2*d, 0) for p in edge2]
edge4.reverse()

tile1 = translate(
    scale(Polygon(edge1 + edge2 + edge3 + edge4), 0.5, 0.5), -d/2, -d/2)
tile2 = translate(tile1, d, 0)
tile3 = translate(tile2, 0, d)
tile4 = translate(tile3, -d, 0)

tile = geopandas.GeoDataFrame(
    geometry = geopandas.GeoSeries([tile1, tile2, tile3, tile4]),
    data = {"element_id": list("abcd")}, crs = ak.crs)

ax = tile.plot(column = "element_id", edgecolor = "w")
tile_base.plot(ax = ax, facecolor = "#0000", edgecolor = "k")

In [ ]:
unit = TileUnit(crs = 2193, spacing = d)
unit.elements = tile
unit.tile = tile_base

In [ ]:
tiling = Tiling(unit, ak, id_var = "DZ2018")
t30 = tiling.get_tiled_map(rotation = 30)

In [ ]:
fig = pyplot.figure(figsize = (24, 15))
ax = fig.add_subplot(111)
ak.plot(ax = ax, facecolor = "#006633", linewidth = 0)

strands = set(t30.element_id)
cols = [x for x in t30.columns if "Rank" in x]
cmaps = ("viridis", "plasma_r", "Spectral", "PRGn")
cmaps = ("Greys", "Greys", "Greys", "Greys")

for strand, col, cmap in zip(strands, cols, cmaps):
    t30[t30.element_id == strand].plot(ax = ax, column = col, cmap = cmap, k = 100, scheme = "equal_interval", linewidth = 0)


## Concentric annuli

In [ ]:
d = 200
c = [Point((0, 0)).buffer(r) for r in range(d, 0, -d//7)]
annuli = [circ.difference(c[i + 1]) for i, circ in enumerate(c[:-1])] + c[-1:]

tile = geopandas.GeoDataFrame(
    geometry = geopandas.GeoSeries(annuli),
    crs = ak.crs, data = {"element_id": list("abcdefg")}
)

In [ ]:
rect_base = TileUnit(crs = 2193, spacing = d)
rect_base.elements = tile
ax = rect_base.elements.plot(column = "element_id")
rect_base.tile.plot(ax = ax, alpha = 0.35)

In [ ]:
hex_base = TileUnit(crs = 2193, shape = "hexagon", spacing = d)
hex_base.elements = tile
ax = hex_base.elements.plot(column = "element_id")
hex_base.tile.plot(ax = ax, alpha = 0.35)

In [ ]:
tri_d_base = TileUnit(crs = 2193, shape = "triangle", 
                    to_hex = False, spacing = d * 2 / 3)
tri_d_base.elements = tile
ax = tri_d_base.elements.plot(column = "element_id")
tri_d_base.tile.plot(ax = ax, alpha = 0.35)

In [ ]:
# tiling = Tiling(tri_d_base, ak, id_var = "DZ2018")
# tiling = Tiling(rect_base, ak, id_var = "DZ2018")
tiling = Tiling(hex_base, ak, id_var = "DZ2018")

In [ ]:
t30 = tiling.get_tiled_map()

In [ ]:
fig = pyplot.figure(figsize = (24, 15))
ax = fig.add_subplot(111)
# ak.plot(ax = ax, facecolor = "#009966", linewidth = 0)

strands = set(t30.element_id)
cols = [x for x in t30.columns if "Rank" in x]
cmaps = ("BrBG", "PiYG", "PRGn", "PuOr", "RdGy", "RdYlBu", "Spectral")

for strand, col, cmap in zip(strands, cols, cmaps):
    t30[t30.element_id == strand].plot(ax = ax, column = col, cmap = cmap, k = 100, scheme = "equal_interval", linewidth = 0)
